

Experimenting with a bunch more MCP Servers

In [11]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os
from IPython.display import Markdown, display
from datetime import datetime
load_dotenv(override=True)

True

In [12]:
import os

# Use the WSL node+npx from nvm
NODE_BIN = "/home/mpantsuyash84270/.nvm/versions/node/v22.21.1/bin"

# Put WSL Node path at the front of PATH
os.environ["PATH"] = NODE_BIN + ":" + os.environ["PATH"]

# Optional checks
print("node:", os.popen("node --version").read().strip())
print("npx :", os.popen("npx --version").read().strip())


node: v22.21.1
npx : 10.9.4


In [13]:
params = {"command": "npx","args": ["-y", "mcp-memory-libsql"],"env": {"LIBSQL_URL": "file:./memory/SS.db"}}

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()

mcp_tools

[Tool(name='create_entities', title='Create new entities with observations', description='Create new entities with observations', inputSchema={'$schema': 'http://json-schema.org/draft-07/schema#', 'type': 'object', 'properties': {'entities': {'type': 'array', 'items': {'type': 'object', 'properties': {'name': {'type': 'string'}, 'entityType': {'type': 'string'}, 'observations': {'type': 'array', 'items': {'type': 'string'}}}, 'required': ['name', 'entityType', 'observations']}}}, 'required': ['entities']}, outputSchema=None, icons=None, annotations=None, meta=None),
 Tool(name='search_nodes', title='Search for entities and their relations using text search with relevance ranking', description='Search for entities and their relations using text search with relevance ranking', inputSchema={'$schema': 'http://json-schema.org/draft-07/schema#', 'type': 'object', 'properties': {'query': {'type': 'string'}, 'limit': {'type': 'number'}}, 'required': ['query']}, outputSchema=None, icons=None, 

In [27]:
instructions = "You use your entity tools as a persistent memory to store and recall information about your conversations."
request = "My name's Suyash. I'm an LLM engineer. I'm learning a course about AI Agents, including the incredible MCP protocol. \
MCP is a protocol for connecting agents with tools, resources and prompt templates, and makes it easy to integrate AI agents with capabilities."
model = "gpt-4.1-mini"

In [15]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Nice to meet you, Ed! It sounds like your course on AI Agents and the MCP protocol is very interesting and valuable for those learning about integrating AI agents with various capabilities. If you'd like, I can help you gather resources, craft explanations, or create examples related to AI Agents or the MCP protocol for your course. Just let me know what you need!

In [16]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, "My name's Ed. What do you know about me?")
    display(Markdown(result.final_output))

I don't have any information about you, Ed, yet. If you'd like, you can tell me more about yourself or share anything you'd like me to remember.

### Check the trace:

https://platform.openai.com/traces

### The 2nd type of MCP server - runs locally, calls a web service



In [17]:
env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}
params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": env}

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()

mcp_tools

[Tool(name='brave_web_search', title=None, description='Performs a web search using the Brave Search API, ideal for general queries, news, articles, and online content. Use this for broad information gathering, recent events, or when you need diverse web sources. Supports pagination, content filtering, and freshness controls. Maximum 20 results per request, with offset for pagination. ', inputSchema={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'Search query (max 400 chars, 50 words)'}, 'count': {'type': 'number', 'description': 'Number of results (1-20, default 10)', 'default': 10}, 'offset': {'type': 'number', 'description': 'Pagination offset (max 9, default 0)', 'default': 0}}, 'required': ['query']}, outputSchema=None, icons=None, annotations=None, meta=None),
 Tool(name='brave_local_search', title=None, description="Searches for local businesses and places using Brave's Local Search API. Best for queries related to physical locations, businesses, re

In [18]:
instructions = "You are able to search the web for information and briefly summarize the takeaways."
request = f"Please research the latest news on Amazon stock price and briefly summarize its outlook. \
For context, the current date is {datetime.now().strftime('%Y-%m-%d')}"
model = "gpt-4o-mini"

In [19]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

As of December 10, 2025, Amazon's stock (AMZN) trades around $229.53 per share. Here's a brief summary of the latest outlook:

1. **Price Targets**: Wall Street analysts have a median one-year price target of **$295.63**, indicating a potential upside of approximately **29.7%** from the current price, supported by a “Strong Buy” rating from 44 analysts.

2. **Bearish Sentiment**: However, some forecasts, including those from CoinCodex, suggest a decline over the short term, predicting the stock could drop to **$213.69** by early January 2026.

3. **Market Sentiment**: The current market sentiment is mixed, with some indicators showing bearish trends despite strong analyst support for the long-term potential of the stock.

4. **AI Infrastructure**: Amazon continues to be central in the AI infrastructure sector, which may contribute positively to its stock outlook as further developments unfold.

Overall, while there are strong long-term forecasts for Amazon, short-term predictions are cautionary, suggesting potential volatility.

### As usual, check out the trace:

https://platform.openai.com/traces

## And now the third type: running remotely

It's actually really hard to find a "remote MCP server" aka "hosted MCP server" aka "managed MCP server".

It's not a common model for using or sharing MCP servers, and there isn't a standard way to discover remote MCP servers.




# And back to the 2nd type: the Polygon.io MCP Server

## NEW SECTION: Introducing polygon.io

Polygon.io is a hugely popular financial data provider. It has a free plan and a paid plan. And it also has an MCP Server!

First, read up on polygon.io on their excellent website, including looking at their pricing:

https://polygon.io

### Polygon.io Part 1: Polygon.io free service 
`POLYGON_API_KEY=xxxx`

In [20]:
load_dotenv(override=True)
polygon_api_key = os.getenv("POLYGON_API_KEY")
if not polygon_api_key:
    print("POLYGON_API_KEY is not set")

In [21]:
from polygon import RESTClient
client = RESTClient(polygon_api_key)
client.get_previous_close_agg("AAPL")[0]

PreviousCloseAgg(ticker='AAPL', close=278.78, high=279.75, low=276.44, open=277.75, timestamp=1765400400000, volume=33038108.0, vwap=278.4798)

### Wrapped into a python module that caches end of day prices

I've made a python module `market.py` that uses this API to look up share prices.

But the free API is quite heavily rate limited - so I've been a bit sneaky; when you ask for a share price, this function retrieves the entire end-of-day equity market, and caches it in our database.


In [22]:
from market import get_share_price
get_share_price("AAPL")

278.78

In [23]:
# no rate limiting concerns!

for i in range(1000):
    get_share_price("AAPL")
get_share_price("AAPL")

278.78

### And I've made this into an MCP Server

Just as we did with accounts.py; see `market_server.py`

In [24]:
params = {"command": "uv", "args": ["run", "market_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    mcp_tools = await server.list_tools()
mcp_tools

[Tool(name='lookup_share_price', title=None, description='This tool provides the current price of the given stock symbol.\n\n    Args:\n        symbol: the symbol of the stock\n    ', inputSchema={'properties': {'symbol': {'title': 'Symbol', 'type': 'string'}}, 'required': ['symbol'], 'title': 'lookup_share_priceArguments', 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': 'lookup_share_priceOutput', 'type': 'object'}, icons=None, annotations=None, meta=None)]

### Let's try it out!

Hopefully gpt-4o-mini is smart enough to know that the symbol for Apple is AAPL

In [25]:
instructions = "You answer questions about the stock market."
request = "What's the share price of Apple?"
model = "gpt-4.1-mini"

async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

The current share price of Apple (AAPL) is $278.78.